# Accurity Glossary Mapping Automation

In [1]:
import os

import pandas as pd
import numpy as np
import sqlalchemy as sql
import requests
import textdistance as td
from IPython import get_ipython
from IPython.display import display, Image

import accurity.nbconfig
# from accurity.sql.adventure_works import s_tables, s_columns
from accurity.tfidfmapping import TfidfDistanceMapping
from accurity.textmapping import TextDistanceMapping

%reload_ext autoreload
%autoreload 2
%matplotlib inline

## 1. Adventure Works (AW) 

In [ ]:
engine = sql.create_engine(os.environ["ADVENTURE_WORKS_DB"])

In [ ]:
tables = (
    pd.read_sql(s_tables, engine)
    .rename(columns={"TABLE_SCHEMA": "table_schema", "TABLE_NAME": "table_name"})
    .reset_index(drop=True)
)

columns = (
    pd.read_sql(s_columns, engine)
    .rename(columns={"TABLE_SCHEMA": "schema", "TABLE_NAME": "table",
            "COLUMN_NAME": "column","IS_NULLABLE": "is_nullable",
            "DATA_TYPE": "type"})
    .assign(table_column=lambda x: x.table + "." + x.column)
    .reset_index(drop=True)
    .sort_values(by=["schema", "table", "column"])
)
columns = columns[~columns.schema.str.startswith("dbo")]

In [ ]:
business_terms = pd.read_csv("./data/AW/business_terms.csv")
business_terms.head()

In [ ]:
columns.head()

In [ ]:
jw_dist_candidates = find_mapping_candidates(sequences=columns.table_column,candidates=business_terms.business_term,
                                             distance_func=jaro_winkler, reverse=-1,
                                             N=5, return_type="df", sequence_name="column", candidate_name="bt",
                                             fpath='./data/mapping_candidates.xlsx')
jw_dist_candidates.head()

In [ ]:
lev_dist_candidates = find_mapping_candidates(sequences=columns.table_column,
                                              candidates=business_terms.business_term, distance_func=levenshtein,
                                              N=5, return_type="df", sequence_name="column", candidate_name="bt",
                                              fpath='./data/mapping_candidates.xlsx')
lev_dist_candidates.head()

In [ ]:
dlev_dist_candidates = find_mapping_candidates(sequences=columns.table_column,candidates=business_terms.business_term,
                                              distance_func=damerau_levenshtein, reverse=1,
                                              N=5, return_type="excel", sequence_name="column", candidate_name="bt",
                                              fpath='./data/mapping_candidates.xlsx')
dlev_dist_candidates.head()

In [ ]:
dlev_dist_candidates = find_mapping_candidates(sequences=columns['table_column'], candidates=business_terms.business_term,
                                              distance_func=needleman_wunsch, reverse=-1,
                                              N=5, return_type="excel", sequence_name="column", candidate_name="bt",
                                              fpath='./data/mapping_candidates.xlsx')
dlev_dist_candidates.head()

In [ ]:
edit_algorithms = [td.jaro_winkler, td.jaro, td.levenshtein, td.damerau_levenshtein]
for algo in edit_algorithms:
    test.find_text_mapping(algorithm=algo, ngrams=1)
    test.save_text_mapping_to_xls(algorithm=algo)

## 2. Groupe Société Générale (BRD)

### 2.1 Text-Distance Mapping Algorithms

#### 2.1.1 Load data for BRD object (`ncandidates=5`)

Initialise new or load from binary file stored `BusinessTermsMapping` object for BRD dataset.

In [34]:
brd = TextDistanceMapping(name='BRD', raw_data_dir='../data/raw/BRD', validation=True)
brd

TextDistanceMappings(name=BRD, validation=True, model_file=../models/text_mapping_brd.bin, raw_data_dir=../data/raw/BRD, output_data_dir=../data/output/, ncandidates=5)

Refresh loading input data for running the mapping algorithm from files `data_fields.csv`, `business_terms.csv`, as well as validation data for validating identifed mappings from file `data_mappings.csv`.

In [35]:
brd.initialize_data()

#### 2.1.2 Finding mapping candidates using edit-based algorithms

Run `edit-based distance` algorithms to find best business terms candidates for each data field. Then, save identified mapping candidates into respective excel sheet and validate the mapping candidates versus the available validation mappings prepared by the client.

In [ ]:
edit_algorithms = [td.jaro_winkler, td.jaro, td.levenshtein, td.damerau_levenshtein]
for algo in edit_algorithms:
    brd.find_save_mapping(algorithm=algo, ngrams=1)
    
brd.find_save_mapping(algorithm=td.jaro_winkler, ngrams=3)

#### 2.1.3 Finding mapping candidates using token-based algorithms

Run `token-based distance` algorithms to find best business terms candidates for each data field. Then, save identified mapping candidates into respective excel sheet and validate the mapping candidates versus the available validation mappings prepared by the client.

In [ ]:
token_algorithms = [td.overlap, td.tversky, td.sorensen, td.cosine, td.jaccard]
for algo in token_algorithms:
    brd.find_save_mapping(algorithm=algo, ngrams=1)

#### 2.1.4 Validation results

In [ ]:
brd.print_validation_results()

### 2.2 TF-IDF Distance Mapping Algorithms

#### 2.2.1 Load data for BRD object (`ncandidates=5`)

In [ ]:
# brd_tfidf = TfidfDistanceMapping(name='BRD', raw_data_dir='../data/raw/BRD', validation=True)
brd_tfidf = TfidfDistanceMapping.load_model(model_file='../models/tfidf_mapping_brd.bin')
brd_tfidf.initialize_data()

#### 2.2.2 Finding mapping candidates using cosine similarity and vectorization algorithm based on TF-IDF weighting and different tokenization methods

In [ ]:
brd_tfidf.find_save_mapping(ngrams=3, analyzer='char', use_stemmer=True)
brd_tfidf.find_save_mapping(ngrams=3, analyzer='char_wb', use_stemmer=True)
brd_tfidf.find_save_mapping(ngrams=1, analyzer='word', use_stemmer=True)
brd_tfidf.find_save_mapping(ngrams=3, analyzer='word', use_stemmer=True)

brd_tfidf.find_save_mapping(ngrams=3, analyzer='char', use_stemmer=False)
brd_tfidf.find_save_mapping(ngrams=3, analyzer='char_wb', use_stemmer=False)
brd_tfidf.find_save_mapping(ngrams=1, analyzer='word', use_stemmer=False)
brd_tfidf.find_save_mapping(ngrams=3, analyzer='word', use_stemmer=False)

#### 2.1.3 Validation results

In [ ]:
brd_tfidf.print_validation_results()

## 3. Bank of Ireland (BOI)

### 3.1 Text-Distance Mapping Algorithms

#### 3.1.1 Load data for BOI object (`ncandidates=5`)

Initialise new or load from binary file stored `BusinessTermsMapping` object for BOI dataset.

In [ ]:
# boi = TextDistanceMapping(name='BOI', raw_data_dir='../data/raw/BOI', validation=True, sep='; ')
boi = TextDistanceMapping.load_model(model_file='../models/text_mapping_boi.bin')
boi

Refresh loading input data for running the mapping algorithm from files `data_fields.csv`, `business_terms.csv`, as well as validation data for validating identifed mappings from file `data_mappings.csv`.

In [ ]:
boi.initialize_data()

#### 3.1.2 Finding mapping candidates using edit-based algorithms

Run `edit-based distance` algorithms to find best business terms candidates for each data field. Then, save identified mapping candidates into respective excel sheet and validate the mapping candidates versus the available validation mappings prepared by the client.

In [ ]:
# edit_algorithms = [td.jaro_winkler, td.jaro, td.levenshtein, td.damerau_levenshtein]
# for algo in edit_algorithms:
#     boi.find_save_mapping(algorithm=algo, ngrams=1)
    
boi.find_save_mapping(algorithm=td.jaro_winkler, ngrams=3)

#### 3.1.3 Finding mapping candidates using token-based algorithms

Run `token-based distance` algorithms to find best business terms candidates for each data field. Then, save identified mapping candidates into respective excel sheet and validate the mapping candidates versus the available validation mappings prepared by the client.

In [ ]:
token_algorithms = [td.overlap, td.tversky, td.sorensen, td.cosine, td.jaccard]
for algo in token_algorithms:
    boi.find_save_mapping(algorithm=algo, ngrams=1)

#### 3.1.4 Validation results

In [ ]:
boi.print_validation_results()

### 3.2 TF-IDF Distance Mapping Algorithms

#### 3.2.1 Load data for BOI object (`ncandidates=5`)

In [ ]:
# boi_tfidf = TfidfDistanceMapping(name='BOI', raw_data_dir='../data/raw/BOI', validation=True, sep='; ')
boi_tfidf = TfidfDistanceMapping.load_model(model_file='../models/tfidf_mapping_boi.bin')
boi_tfidf.initialize_data()

#### 3.2.2 Finding mapping candidates using cosine similarity and vectorization algorithm based on TF-IDF weighting and different tokenization methods

In [ ]:
boi_tfidf.find_save_mapping(ngrams=3, analyzer='char', use_stemmer=True)
boi_tfidf.find_save_mapping(ngrams=3, analyzer='char_wb', use_stemmer=True)
boi_tfidf.find_save_mapping(ngrams=1, analyzer='word', use_stemmer=True)
boi_tfidf.find_save_mapping(ngrams=3, analyzer='word', use_stemmer=True)

boi_tfidf.find_save_mapping(ngrams=3, analyzer='char', use_stemmer=False)
boi_tfidf.find_save_mapping(ngrams=3, analyzer='char_wb', use_stemmer=False)
boi_tfidf.find_save_mapping(ngrams=1, analyzer='word', use_stemmer=False)
boi_tfidf.find_save_mapping(ngrams=3, analyzer='word', use_stemmer=False)

#### 3.2.3 Validation results

In [ ]:
boi_tfidf.print_validation_results()

In [ ]:
response = requests.get("https://http.cat/100")

print(response.headers.get("Content-Type"))
with open("goat.jpeg", "wb") as f:
    f.write(response.content)

display(Image(filename='goat.jpeg'))

In [ ]:
def urlify(in_string):
    return in_string.replace(' ', '%20')

base_url = "http://127.0.0.1:8001/api/1.0/mappings"
data_field = 'account appl rel type.account appl rel type cd'

people_endpoint = os.path.join(base_url, urlify(data_field))
headers = {'X-API-Key': 'btmappings'}

r = requests.get(url=people_endpoint, headers=headers)
r.json()[data_field]


# r = requests.get(url=people_endpoint, headers=headers)
# print(r.status_code, r.reason, r.json()[0])

In [ ]:
a = [1, 2, 3, 3, 2, 1]

def longestPeak(a):
    maxpeak = 0
    peak = 0
    asc, desc = False, False
    
    for i in range(2, len(a)):
        print(f"Befor i:{i:>2} => {a[i]:>2}, peak:{peak}, maxpeak:{maxpeak}")
        if a[i-2] < a[i-1] < a[i]:
            if i == 2:
                peak += 2
            else:
                peak += 1
            asc = True
        if a[i-2] > a[i-1] < a[i]:
            if peak + 1 > maxpeak and asc and desc:
                maxpeak = peak + 1
            peak = 1
            asc = True
        if a[i-2] > a[i-1] == a[i]:
            if peak + 1 > maxpeak and asc and desc:
                maxpeak = peak + 1
            peak = 0
            asc = 0
            desc = 0
        if a[i-2] > a[i-1] > a[i]:
            peak += 1
            desc = True
            if peak + 1 > maxpeak and asc and desc:
                maxpeak = peak + 1
        if a[i-2] < a[i-1] > a[i]:
            if i == 2:
                peak += 2
            else:
                peak += 1
            desc = True
            asc = True
            if peak + 1 > maxpeak and asc and desc:
                maxpeak = peak + 1
        if a[i-1] == a[i]:
            if peak + 1 > maxpeak and asc and desc:
                maxpeak = peak + 1
            peak = 0
            asc = 0
            desc = 0
        if a[i-2] == a[i-1] < a[i]:
            peak += 1
            asc = True
        print(f"After i:{i:>2} => {a[i]:>2}, peak:{peak}, maxpeak:{maxpeak}\n")
    return maxpeak

longestPeak(a)

In [ ]:
intervals = [[1, 2], [3, 5], [4, 7], [6, 8], [9, 10]]
intervals = [[89, 90], [-10, 20], [-50, 0], [70, 90], [90, 91], [90, 95]]
intervals = [[1, 22], [-20, 30]]

def merge_intervals(intervals):
    intervals.sort(key=lambda x: (x[0], x[1]))
    print(intervals)
    
    result = []
    current = intervals[0]
    
    i = 1    
    while i < len(intervals):
        if current[1] >= intervals[i][0]:
            current[1] = max(intervals[i][1], current[1])
        else:
            result.append(current)
            current = intervals[i]
        if i == len(intervals) - 1:
            result.append(current)
        print(i, current)
        i += 1
    return result

merge_intervals(intervals)

In [ ]:
array = [1, 2, 4, 7, 10, 11, 7, 12, 6, 7, 16, 18, 19]

def subarraysort(array):
    sorted_array = sorted(array)
    for i in range(len(array)):
        if array[i] != sorted_array[i]:
            start = i
            break
    for i in reversed(range(len(array))):
        if array[i] != sorted_array[i]:
            end = i
            break
    return start, end

subarraysort(array)

In [ ]:
array = [1, 11, 3, 0, 15, 5, 2, 4, 10, 7, 12, 6]
array = [1, 2, 3, 4]
array = [19, -1, 18, 17, 2, 10, 3, 12, 5, 16, 4, 11, 8, 7, 6, 15, 12, 12, 2, 1, 6, 13, 14]

def largestRange(array):
    array.sort()
    maxstart, maxend = array[0], array[0]
    start, end = maxstart, maxend
    for i in range(1, len(array)):
        if array[i] - array[i-1] <= 1:
            if array[i] - array[i-1] == 1:
                end += 1
            if i == len(array) - 1:
                if end - start > maxend - maxstart:
                    return start, end
                else:
                    return maxstart, maxend
        else:
            if end - start > maxend - maxstart:
                maxstart = start
                maxend = end
            start, end = array[i], array[i]
            
    return maxstart, maxend

largestRange(array)

In [ ]:
import requests
r = requests.get("http://127.0.0.1:8000/api/1.0/mappings", headers={'X-API-Key': 'btmappings'})
r.json()

In [115]:
### from slugify import slugify
from typing import List


class String:

    def __init__(self, raw: str, idx: int) -> None:
        self._raw = raw
        self._idx = idx

    def __repr__(self):
        return f"String(idx={self._idx}, raw='{self._raw}', slugified='{self.slugified}')"

    @property
    def slugified(self):
        return slugify(self._raw, separator=" ")


class StringCatalog:

    name = 'StringCatalog'

    def __init__(self) -> None:
        self._strings = set()

    def __len__(self) -> None:
        return len(self._strings)

    def __repr__(self) -> None:
        return f"{self.name} with {len(self)} string(s)"

    def as_pandas(self) -> None:
        return pd.Series([s._raw for s in self._strings], name=self.name)

    def add_strings(self, strings: List[str]) -> None:
        for string in strings:
            self._strings.add(String(raw=string, idx=len(self)))

    def get_string(self, idx: int = None, raw: str = None) -> String:
        for string in self._strings:
            if string._raw == raw or string._idx == idx:
                return string


class InputStringCatalog(StringCatalog):
    name = 'InputStringCatalog'
    
class MatchingCandidateCatalog(StringCatalog):
    name = 'MatchingStringCatalog'
    

input_catalog = InputStringCatalog()
input_catalog.add_strings(strings=['entity_name.attribute_name0', 'entity_name.attribute_name1', 'entity_name.attribute_name2'])
input_catalog.get_string(idx=2), input_catalog.get_string(raw='entity_name.attribute_name2')
input_catalog.as_pandas()

candidate_catalog = MatchingCandidateCatalog()
candidate_catalog.add_strings(strings=['data_field.data_structure', 'data_field.data_structure', 'data_field.data_structure'])
candidate_catalog.as_pandas()

0    data_field.data_structure
1    data_field.data_structure
2    data_field.data_structure
Name: MatchingStringCatalog, dtype: object

In [136]:
a = pd.DataFrame(data={'A': [1,2,2], 'B': [3,3,3]})
a[a.columns[0]]

0    1
1    2
2    2
Name: A, dtype: int64